In [1]:
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn import metrics
from contextlib import redirect_stdout
from itertools import compress
import scipy.sparse as sparse
import lightgbm as lgb
import pandas as pd
import numpy as np
import tqdm
import os
import gc
import sys
sys.path.append('../code/pipeline/')
sys.path.append('../code/utils/')
sys.path.append('../code/')
import data_pipeline as dp
import data_utils as du
import perf_utils as pu
import eval_utils as eu
import io_utils as iu
import config

In [2]:
pairs = [# ('productId', 'LBS'),  # no need in this case
         ('productType', 'kw1'),  # 'kw1' looks very overfitting prone, to be decide whether to keep it
         ('productType', 'kw2'),
         ('productType', 'kw3'),
         ('productType', 'topic1'),
         ('aid', 'topic2'),
         ('productType', 'topic2'),
         # ('productType', 'topic3'),  # might help in predicting negative samples
         # ('productType', 'appIdInstall'),  # might help in predicting negative samples
         # ('productType', 'appIdAction'),  # might help in predicting negative samples
         ('advertiserId', 'interest1'),
         ('aid', 'interest2'),
         ('creativeSize', 'interest2'), 
         ('campaignId', 'interest4'),  # whether to keep it? 
         ('aid', 'interest5'),  
         ('aid', 'ct'),
         ('aid', 'os')]

In [3]:
# ================
# Data Preparation
# ================
# instantiate data loader
ckr_loaders = (dp.ListClickrateDataManager.build_data(ad_feat_name, user_feat_name) 
               for ad_feat_name, user_feat_name in pairs)
union_loader = dp.DataUnion(*ckr_loaders)

with pu.profiler("loading training data"):
    cols, X_tv = union_loader.load("train")
    X_tv = sparse.csr_matrix(X_tv)
    gc.collect()

print("Train Data Shape: {}".format(X_tv.shape))
print("Train Column Numbers: {}".format(len(cols)))

[10:14:05] Finish loading training data. △M: +9.66GB. △T: 2.1 minutes.
Train Data Shape (Embedding): (8798814, 156)
Train Column Numbers (Embedding): 156


In [4]:
df_train = du.load_raw_data("train")
y = df_train['label'].values.copy()
y = (y + 1) / 2  # -1, 1 -> 0, 1

In [5]:
n_splits = 3
sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=1 / 3, random_state=20180505)  # for reproducibility
split_indices = [(train_index, valid_index) for train_index, valid_index in sss.split(df_train, y)]

# n_splits = 5  # use 3 instead of 5 to save time
# skf = StratifiedKFold(n_splits=n_splits)
# split_indices = [(train_index, valid_index) for train_index, valid_index in skf.split(df_train, y)]

In [6]:
aids = df_train['aid'].values
with pu.profiler("splitting train/valid set"):
    train_index, valid_index = split_indices[0]
    X_train, X_valid = X_tv[train_index, :], X_tv[valid_index, :]
    y_train, y_valid = y[train_index], y[valid_index]
    aids_train, aids_valid = aids[train_index], aids[valid_index]
    assert X_train.shape[0] + X_valid.shape[0] == X_tv.shape[0]
    
    del X_tv
    gc.collect()

print("Training Set Size: {}".format(X_train.shape))
print("Validation Set Size: {}".format(X_valid.shape))

[10:15:03] Finish splitting train/valid set. △M: +145.46MB. △T: 20.5 seconds.
Training Set Size: (5865876, 156)
Validation Set Size: (2932938, 156)


In [7]:
with pu.profiler("preparing LightGBM data"):
    # lgb_train = lgb.Dataset(X_train.astype(np.float32), y_train, feature_name=cols_train)  # cause bugs I don't know how to fix
    # lgb_valid = lgb.Dataset(X_valid.astype(np.float32), y_valid, feature_name=cols_train)
    lgb_train = lgb.Dataset(X_train.astype(np.float32), y_train)
    lgb_valid = lgb.Dataset(X_valid.astype(np.float32), y_valid)
    gc.collect()

[10:15:15] Finish preparing LightGBM data. △M: +9.64GB. △T: 7.1 seconds.


In [8]:
with pu.profiler("releasing memory"):
    del X_train
    # del X_valid
    gc.collect()

[10:15:21] Finish releasing memory. △M: -6.42GB. △T: 0.7 seconds.


In [9]:
log_folder = os.path.join(config.LOG_DIR, 'lgbm/pipeline/0520/')
log_file = 'v1.log'
log_path = os.path.join(log_folder, log_file)
os.makedirs(log_folder, exist_ok=True)

In [10]:
# v2 parameters
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'max_depth': 15,
    'num_leaves': 120,
    'learning_rate': 0.15,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'verbose': 0
}
num_rounds = 1000
# # v3 parameters
# params = {
#     'boosting_type': 'gbdt',
#     'objective': 'binary',
#     'metric': 'auc',
#     'max_depth': 6,
#     'num_leaves': 64,
#     'learning_rate': 0.1,
#     'feature_fraction': 0.5,
#     'bagging_fraction': 0.5,
#     'verbose': 0
# }
# num_rounds = 5000
eval_auc = eu.build_lightgbm_online_auc_eval(aids_train, aids_valid)

with iu.DuplicatedLogger(log_path):
    lgbm = lgb.train(params,
                     lgb_train,
                     num_boost_round=num_rounds,
                     valid_sets=[lgb_train, lgb_valid], 
                     valid_names=['train', 'valid1'],
                     feval = eval_auc,
                     early_stopping_rounds=50)

[1]	train's auc: 0.692953	train's online_auc: 0.687775	valid1's auc: 0.691607	valid1's online_auc: 0.685972
Training until validation scores don't improve for 50 rounds.
[2]	train's auc: 0.699903	train's online_auc: 0.695142	valid1's auc: 0.697983	valid1's online_auc: 0.692608
[3]	train's auc: 0.703007	train's online_auc: 0.69918	valid1's auc: 0.701009	valid1's online_auc: 0.696521
[4]	train's auc: 0.703966	train's online_auc: 0.700638	valid1's auc: 0.702061	valid1's online_auc: 0.698048
[5]	train's auc: 0.70518	train's online_auc: 0.701501	valid1's auc: 0.703077	valid1's online_auc: 0.6989
[6]	train's auc: 0.706284	train's online_auc: 0.703153	valid1's auc: 0.704019	valid1's online_auc: 0.700354
[7]	train's auc: 0.707243	train's online_auc: 0.704188	valid1's auc: 0.704794	valid1's online_auc: 0.701204
[8]	train's auc: 0.708071	train's online_auc: 0.70507	valid1's auc: 0.70558	valid1's online_auc: 0.701991
[9]	train's auc: 0.708469	train's online_auc: 0.70543	valid1's auc: 0.705922	val

[76]	train's auc: 0.744629	train's online_auc: 0.74777	valid1's auc: 0.726725	valid1's online_auc: 0.720731
[77]	train's auc: 0.744882	train's online_auc: 0.748074	valid1's auc: 0.726707	valid1's online_auc: 0.720727
[78]	train's auc: 0.74518	train's online_auc: 0.748722	valid1's auc: 0.726754	valid1's online_auc: 0.720714
[79]	train's auc: 0.74555	train's online_auc: 0.74903	valid1's auc: 0.726732	valid1's online_auc: 0.7207
[80]	train's auc: 0.745953	train's online_auc: 0.749379	valid1's auc: 0.726817	valid1's online_auc: 0.720746
[81]	train's auc: 0.746329	train's online_auc: 0.74945	valid1's auc: 0.726769	valid1's online_auc: 0.720743
[82]	train's auc: 0.746673	train's online_auc: 0.749964	valid1's auc: 0.726876	valid1's online_auc: 0.720953
[83]	train's auc: 0.747076	train's online_auc: 0.750631	valid1's auc: 0.726987	valid1's online_auc: 0.721066
[84]	train's auc: 0.747369	train's online_auc: 0.750942	valid1's auc: 0.726973	valid1's online_auc: 0.721083
[85]	train's auc: 0.747518

In [11]:
# these will cause LightGBM bug I don't know how to fix. Just skip it
log_file = 'v1.feature_importance.csv'
log_path = os.path.join(log_folder, log_file)

df_feature_importance = pd.DataFrame({"feature": cols, "importance": lgbm.feature_importance()})
df_feature_importance = df_feature_importance.sort_values("importance", ascending=False)
df_feature_importance.to_csv(log_path, index=False)
df_feature_importance.head(30)

,feature,importance
88,bsClickrate@aid_x_interest2_q100,295
72,bsClickrate@advertiserId_x_interest1_q0,216
100,bsClickrate@creativeSize_x_interest2_q100,199
120,bsClickrate@aid_x_interest5_q0,197
104,impression@creativeSize_x_interest2_mean,183
76,bsClickrate@advertiserId_x_interest1_q100,179
16,bsClickrate@productType_x_kw2_q100,164
124,bsClickrate@aid_x_interest5_q100,162
18,bsClickrate@productType_x_kw2_mean,152
17,bsClickrate_weighted_avg@productType_x_kw2,145


In [12]:
with pu.profiler("making prediction on validation set"):
    df_valid = df_train.iloc[valid_index]
    proba_valid = lgbm.predict(X_valid.astype(np.float32))

log_file = 'v1.online_auc.csv'
log_path = os.path.join(log_folder, log_file)
df_score = eu.online_auc(df_valid['aid'], y_valid, proba_valid, ret_verbose=True)
df_score.rename(columns={'selector': 'aid'}, inplace=True)
df_score = df_score[['aid', 'auc']]  # sort columns
df_score = df_score.sort_values("auc", ascending=False)
df_score.to_csv(log_path, index=False)

[10:34:51] Finish making prediction on validation set. △M: +20.0KB. △T: 8.1 seconds.


In [13]:
online_auc = df_score['auc'].mean()
simple_auc = metrics.roc_auc_score(y_valid, proba_valid)
print("Online AUC: {:.6f}".format(online_auc))
print("Simple AUC: {:.6f}".format(simple_auc))

Online AUC: 0.721145
Simple AUC: 0.727022


In [14]:
with pu.profiler("cleaning memory"):
    del lgb_train
    del lgb_valid
    # del X_train
    del X_valid
    gc.collect()

[10:35:07] Finish cleaning memory. △M: -4.46GB. △T: 0.7 seconds.


In [ ]:
with pu.profiler("loading testing data"):
    _, X_test = union_loader.load("test1")
    X_test = sparse.csr_matrix(X_test)
    gc.collect()

print("Test Data Shape: {}".format(X_test.shape))
# print("Test Column Numbers: {}".format(len(cols_test)))

In [17]:
df_test = du.load_raw_data("test")
X_test = X_test.astype(np.float32)

with pu.profiler("making prediction on testing set"):
    proba_test = lgbm.predict(X_test)
    assert len(proba_test.shape) == 1
    assert proba_test.shape[0] == df_test.shape[0]

[13:42:18] Finish making prediction on testing set. △M: +0B. △T: 5.1 seconds.


In [18]:
subm_folder = '../subm/lgbm/0518_v4'
subm_file = 'submission.csv'
subm_path = os.path.join(subm_folder, subm_file)
os.makedirs(subm_folder, exist_ok=True)

subm = df_test.copy()
subm["score"] = proba_test
subm.to_csv(subm_path, index=False)